**ML - Fall 2023 - Practical - 3**<br>
**Contest**

Here is my code for the contest. I just made an ensemble of couple of models and it worked not that bad.

In [ ]:
!pip install typing-extensions --upgrade
!pip install ydata-profiling
!pip install gdown

In [86]:
# I have certainly imported some things twice or even more @.@
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from ydata_profiling import ProfileReport
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from pandas import read_csv
from pandas.plotting import scatter_matrix
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVR, SVC
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from xgboost import XGBClassifier, XGBRegressor
from sklearn.linear_model import SGDRegressor, BayesianRidge
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score

Loading the train dataset

In [96]:
df = pd.read_csv('train.csv')
df.dropna(inplace=True)

Loading the test dataset

In [97]:
df_test = pd.read_csv("test.csv")

# P3: Pre-processing (15 points)
In this section, we perform some pre-processing to make the data ready for model.

In [98]:
corr = df.corr()
independants = corr["target"][abs(corr["target"])< 0.015]
print(independants)
df.drop(independants.index, axis=1, inplace=True)
df.head()

ID            0.012904

spell_used   -0.000336

Name: target, dtype: float64


<ipython-input-98-7bf5f5b537df>:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.

  corr = df.corr()


,trap_used,character,position,location,assists,damage_amount,towers_damage,rooms_damage,deaths,money_total,kill_death_ratio,kills_total,level,controlling_time,total_damage,earned_damage,special_enemy_killed,room_kills,main_score,target
0,12,TwistedFate,Side.blue,Lane.top_lane,9.0,1862,5119,1862,9,9152,1.222222,2,13,19,99079,25062,149,1,21,False
2,12,Ziggs,Side.blue,Lane.bot_lane,3.0,543,2754,543,4,9014,1.750000,4,12,11,106757,10049,168,0,11,False
3,7,Senna,Side.red,Lane.bot_lane,4.0,1547,1547,1547,6,8155,1.000000,2,12,14,53993,12480,129,1,11,False
5,11,Rengar,Side.red,Lane.jungle,7.0,1564,16053,1564,4,11536,3.500000,7,14,14,128646,40236,27,1,17,True
6,12,Ziggs,Side.blue,Lane.bot_lane,9.0,19665,20431,19665,3,10699,5.333333,7,12,11,123192,8057,140,4,9,True


In [99]:
dft= df_test.drop(independants.index, axis=1)

Label Encoding

In [100]:
le1= preprocessing.LabelEncoder()
le2 = preprocessing.LabelEncoder()
le3 = preprocessing.LabelEncoder()
le4 = preprocessing.LabelEncoder()
le1.fit(df['character'])
le2.fit(df['position'])
le3.fit(df['location'])
df['target'] = le4.fit_transform(df['target'])
df['character'] = le1.transform(df['character'])
df['position'] = le2.transform(df['position'])
df['location'] = le3.transform(df['location'])
df.head()

,trap_used,character,position,location,assists,damage_amount,towers_damage,rooms_damage,deaths,money_total,kill_death_ratio,kills_total,level,controlling_time,total_damage,earned_damage,special_enemy_killed,room_kills,main_score,target
0,12,129,0,3,9.0,1862,5119,1862,9,9152,1.222222,2,13,19,99079,25062,149,1,21,0
2,12,153,0,0,3.0,543,2754,543,4,9014,1.750000,4,12,11,106757,10049,168,0,11,0
3,7,105,1,0,4.0,1547,1547,1547,6,8155,1.000000,2,12,14,53993,12480,129,1,11,0
5,11,99,1,1,7.0,1564,16053,1564,4,11536,3.500000,7,14,14,128646,40236,27,1,17,1
6,12,153,0,0,9.0,19665,20431,19665,3,10699,5.333333,7,12,11,123192,8057,140,4,9,1


In [101]:
dft['character'] = le1.transform(dft['character'])
dft['position'] = le2.transform(dft['position'])
dft['location'] = le3.transform(dft['location'])

normalizing using standard scalar

In [102]:
scaler = StandardScaler()
cols = df.columns.difference(['target'])
scaler.fit(df[cols])
df[cols] = scaler.transform(df[cols])
df.head()

,trap_used,character,position,location,assists,damage_amount,towers_damage,rooms_damage,deaths,money_total,kill_death_ratio,kills_total,level,controlling_time,total_damage,earned_damage,special_enemy_killed,room_kills,main_score,target
0,0.827751,1.135156,-0.985152,0.731026,0.143811,-0.243388,-0.422104,-0.243388,1.079311,-0.493218,-0.631880,-0.857765,-0.348308,0.005235,-0.263822,0.363075,0.336919,-0.024765,-0.438784,0
2,0.827751,1.671426,-0.985152,-1.416887,-0.920181,-0.656293,-0.656259,-0.656293,-0.576446,-0.532949,-0.491786,-0.394431,-0.769585,-0.562991,-0.154760,-1.152349,0.563839,-0.786357,-0.917862,0
3,-0.331804,0.598886,1.015071,-1.416887,-0.742849,-0.341997,-0.775762,-0.341997,0.085857,-0.780259,-0.690867,-0.857765,-0.769585,-0.349906,-0.904240,-0.906962,0.098057,-0.024765,-0.917862,0
5,0.595840,0.464819,1.015071,-0.700916,-0.210853,-0.336675,0.660454,-0.336675,-0.576446,0.193147,-0.027264,0.300569,0.072968,-0.349906,0.156159,1.894751,-1.120144,-0.024765,-0.630415,1
6,0.827751,1.671426,-0.985152,-1.416887,0.143811,5.329738,1.093912,5.329738,-0.907597,-0.047830,0.459378,0.300569,-0.769585,-0.562991,0.078688,-1.353423,0.229431,2.260014,-1.013678,1


In [103]:
dft[cols] = scaler.transform(dft[cols])

# Model

In [104]:
X = df.drop(['target'], axis=1).values
y = df['target'].values
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)

In [105]:
X_test = dft.values

Ensemble Model

In [107]:
base_models = [
    ('rf', RandomForestClassifier(n_estimators=150, max_depth=7)),
    ('svc', make_pipeline(SimpleImputer(strategy='mean'), StandardScaler(), SVC(probability=True))),
    ('xgb', XGBClassifier(n_estimators=150, learning_rate=0.1, max_depth=5)),
    ('lgb', LGBMClassifier(n_estimators=130, learning_rate=0.1, max_depth=2, verbose=0)),
    ('extra_trees', ExtraTreesClassifier(n_estimators=100, max_depth=7)),
    ('ada_boost', AdaBoostClassifier(n_estimators=100, learning_rate=0.1))
]

meta_model = GradientBoostingClassifier(n_estimators=150, learning_rate=0.1, max_depth=5, random_state=42)

stacking_ensemble = StackingClassifier(estimators=base_models, final_estimator=meta_model, stack_method='auto')

stacking_ensemble.fit(X_train, y_train)

y_pred = stacking_ensemble.predict(X_test)
y_val_pred = stacking_ensemble.predict(X_val)
accuracy = accuracy_score(y_val, y_val_pred)
print(f'Stacking Ensemble Model Accuracy: {accuracy:.4f}')


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf

Stacking Ensemble Model Accuracy: 0.8377


In [108]:
def write_test(pred,name):
  newtest = pd.read_csv("test.csv")
  newtest["targetnum"]=pred
  newtest["target"]= newtest["targetnum"]==1
  newtest = newtest[["ID","target"]]
  newtest.to_csv(f"{name}.csv")

In [109]:
write_test(y_pred,"Cocomelon-Model")

Let's go &#129393;